In [18]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB


locations_df = pd.read_csv("locations.csv")
order_list_df = pd.read_excel("order_list.xlsx")
travel_matrix_df = pd.read_csv("travel_matrix.csv")
trucks_df = pd.read_csv("trucks.csv")

depot = 'A123'  # depot
order_destinations = order_list['Destination Code'].tolist()
order_list['Destination Code'] = order_list['Destination Code'].astype(str)  # Ensure data type
location_codes = [depot] + [str(code) for code in order_destinations]
locations_df = locations[locations['location_code'].isin(location_codes)]

In [3]:
# Convert 'trucks_allowed' column from string representation to actual list
locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(eval)
locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(set)
locations_df.set_index('location_code', inplace=True)

# Convert DataFrame to dictionary
locations_dff = locations_df.to_dict(orient='index')

# Get all location codes
# location_codes = list(locations_dff.keys())


C:\Users\chede appliances\AppData\Local\Temp\ipykernel_12272\1834188016.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(eval)
C:\Users\chede appliances\AppData\Local\Temp\ipykernel_12272\1834188016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(set)


In [5]:
# Create a mapping from truck type to truck IDs
type_to_ids = {}
for _, row in trucks.iterrows():
    truck_type = row['truck_type']
    truck_id = row['truck_id']
    if truck_type not in type_to_ids:
        type_to_ids[truck_type] = []
    type_to_ids[truck_type].append(truck_id)


In [6]:
allowed_vehicles = {}
for i in range(len(location_codes)):
    for j in range(i + 1, len(location_codes)):
        loc_i = location_codes[i]
        loc_j = location_codes[j]
        trucks_i = locations_dff[loc_i]['trucks_allowed']
        trucks_j = locations_dff[loc_j]['trucks_allowed']

        # Find common truck types
        common_truck_types = trucks_i.intersection(trucks_j)

        # Convert truck types to truck IDs
        common_truck_ids = []
        for truck_type in common_truck_types:
            common_truck_ids.extend(type_to_ids.get(truck_type, []))

        allowed_vehicles[(loc_i, loc_j)] = common_truck_ids


In [7]:

# Initialize matrices
num_locations = len(location_codes)
distance_matrix = np.zeros((num_locations, num_locations))
time_matrix = np.zeros((num_locations, num_locations))
# Map location codes to indices
location_index = {location_codes[i]: i for i in range(num_locations)}


for _, row in travel_matrix.iterrows():
    if row['source_location_code'] in location_codes and row['destination_location_code'] in location_codes:
        i = location_index[row['source_location_code']]
        j = location_index[row['destination_location_code']]
        distance_matrix[i][j] = row['travel_distance_in_km']
        time_matrix[i][j] = row['travel_time_in_min']

order_list['Destination Code'] = order_list['Destination Code'].astype(str)

# Initialize the demands dictionary with zero demand for all locations
demands_dict = {location_code: 0 for location_code in location_codes}

# Update the demands based on the order_list DataFrame
for _, row in order_list.iterrows():
    destination_code = row['Destination Code']
    demands_dict[destination_code] += row['Total Weight']

demands_dict


{'A123': 0,
 '12854121': 1200.0,
 '12854171': 500.0,
 '12854197': 984.0,
 '12854137': 400.0,
 '12854133': 1035.0,
 '12854159': 240.0,
 '12854127': 480.0,
 '12983715': 1728.0,
 '11950159': 1532.0,
 '12838671': 376.0,
 '11646201': 188.0,
 '10208730': 385.6,
 '10214844': 1135.0,
 '10208746': 440.0,
 '12738762': 1500.0,
 '10208117': 314.88,
 '12171444': 1254.0,
 '13015040': 220.0,
 '13067657': 418.0,
 '10208120': 48.0,
 '13005196': 418.0,
 '12769191': 180.0}

In [8]:
# Initialize the capacity dictionary with zero capacity for all trucks
truck_capacities = {t_id: 0 for t_id in trucks['truck_id']}

# capacity of trucks
for _, row in trucks.iterrows():
    t_id = row['truck_id']
    truck_capacities[t_id] += row['truck_max_weight']


num_vehicles = len(truck_capacities)

In [13]:
# Data Preprocessing
depot = 'A123'  # Define the depot
order_destinations = order_list['Destination Code'].tolist()
order_list['Destination Code'] = order_list['Destination Code'].astype(str)  # Ensure data type
location_codes = [str(depot)] + [str(code) for code in order_destinations]

locations_df = locations[locations['location_code'].isin(location_codes)]
locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(eval)
locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(set)
locations_df.set_index('location_code', inplace=True)
locations_dff = locations_df.to_dict(orient='index')
# location_codes = list(locations_dff.keys())

# Mapping truck types to IDs
type_to_ids = {}
for _, row in trucks.iterrows():
    truck_type = row['truck_type']
    truck_id = row['truck_id']
    if truck_type not in type_to_ids:
        type_to_ids[truck_type] = []
    type_to_ids[truck_type].append(truck_id)

# Define allowed vehicles
allowed_vehicles = {}
for i in range(len(location_codes)):
    for j in range(i + 1, len(location_codes)):
        loc_i = location_codes[i]
        loc_j = location_codes[j]
        trucks_i = locations_dff[loc_i]['trucks_allowed']
        trucks_j = locations_dff[loc_j]['trucks_allowed']
        common_truck_types = trucks_i.intersection(trucks_j)
        common_truck_ids = []
        for truck_type in common_truck_types:
            common_truck_ids.extend(type_to_ids.get(truck_type, []))
        allowed_vehicles[(loc_i, loc_j)] = common_truck_ids

# Initialize matrices
num_locations = len(location_codes)
distance_matrix = np.zeros((num_locations, num_locations))
time_matrix = np.zeros((num_locations, num_locations))
location_index = {location_codes[i]: i for i in range(num_locations)}

for _, row in travel_matrix.iterrows():
    if row['source_location_code'] in location_codes and row['destination_location_code'] in location_codes:
        i = location_index[row['source_location_code']]
        j = location_index[row['destination_location_code']]
        distance_matrix[i][j] = row['travel_distance_in_km']
        time_matrix[i][j] = row['travel_time_in_min']

# Initialize demands
demands_dict = {location_code: 0 for location_code in location_codes}
for _, row in order_list.iterrows():
    destination_code = row['Destination Code']
    demands_dict[destination_code] += row['Total Weight']

# Prepare time windows
time_windows = {location_code: 0 for location_code in location_codes}
time_window = [
    (int(start.split(':')[0]), int(end.split(':')[0]))
    for start, end in zip(locations_df['location_loading_unloading_window_start'], locations_df['location_loading_unloading_window_end'])
]

for idx, value in enumerate(time_window):
    time_windows[location_codes[idx]] = value

C:\Users\chede appliances\AppData\Local\Temp\ipykernel_12272\1605294289.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(eval)
C:\Users\chede appliances\AppData\Local\Temp\ipykernel_12272\1605294289.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df['trucks_allowed'] = locations_df['trucks_allowed'].apply(set)


In [33]:
locations_df.head(3)


,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
location_code,,,,,
12854197,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
13005196,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
12854133,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [19]:
# Convert loading/unloading windows to minutes with explicit format
locations_df['start_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.minute
locations_df['end_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.minute

# Extract relevant data
locations = locations_df['location_code'].tolist()
orders = order_list_df.to_dict(orient='records')
travel_matrix = travel_matrix_df.set_index(['source_location_code', 'destination_location_code']).to_dict(orient='index')
trucks = trucks_df.to_dict(orient='records')

# Constants
service_time_customer = 20  
service_time_depot = 60  

depot1 = "A123"
customers_int=order_list_df["Destination Code"].tolist()
invoice = order_list_df["Invoice No."].tolist()
customers_1 = [str(i) for i in customers_int]
customers=zip(invoice,customers_1)
# customers = locations[:len(locations)-1]
loc_without_depot = locations[: len(locations)-1]
locations_df.set_index('location_code', inplace=True)

C:\Users\chede appliances\AppData\Local\Temp\ipykernel_12272\198644590.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df['start_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.minute
C:\Users\chede appliances\AppData\Local\Temp\ipykernel_12272\198644590.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_df['end_minutes'] = pd.

In [20]:
len(location_codes)

26

In [21]:
for i in range(len(order_destinations)):
    order_destinations[i] =  str(order_destinations[i])

loc_df = pd.Series(location_codes)
loc_df[0]

'A123'

In [22]:
## to ensure that order of location_codes and destination is same( as we are going to use indices)
for i in range(len(location_codes)):
    if i>0:
        
        if str(order_destinations[i-1])==location_codes[i]:
            print('Yes')

        else:
            print('No')

Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes


In [59]:
# model 1

In [23]:
# Create the model
model = gp.Model("CVRPTW2")

# Create decision variables
x = {}
t = {}
y = {}
for k in truck_capacities:
    y[k] = model.addVar(vtype=GRB.BINARY, name=f'y_{k}')
    for i in range(len(location_codes)):
        for j in range(len(location_codes)):
            if i != j: #and k in allowed_vehicles.get((str(loc_df.iloc[i]), str(loc_df.iloc[j])), []):
                x[(i, j, k)] = model.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}_{k}')
        if i in range(len(location_codes)):
            t[(i, k)] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f't_{i}_{k}')

# Objective function: Minimize total distance and fixed costs
obj = gp.quicksum(
    travel_matrix.get((loc_df.iloc[i], loc_df.iloc[j]), {}).get('travel_distance_in_km', 0) * x[(i, j, k)] * (20000 - int(truck_capacities[k]) / 1000)
    for k in truck_capacities
    for i in range(len(location_codes))
    for j in range(len(location_codes)) if (i, j, k) in x and i != 0 and j != 0
) + gp.quicksum(
    int(truck_capacities[k]) * 2 * y[k]
    for k in truck_capacities
)
model.setObjective(obj, GRB.MINIMIZE)

# Flow balancing constraint
for i in range(len(location_codes)):
    model.addConstr(
        gp.quicksum(x[(i, j, k)] for j in range(len(location_codes)) for k in truck_capacities if i != j and (i, j, k) in x and j!=0) ==
        gp.quicksum(x[(j, i, k)] for j in range(len(location_codes)) for k in truck_capacities if i != j and (j, i, k) in x and j!=0),
        f"Flow_Balancing_{i}"
    )

# Demand constraint
for k in truck_capacities:
    truck_max_weight = int(truck_capacities[k])  # Maximum weight capacity of truck k
    model.addConstr(
        gp.quicksum(
            int(order_list['Total Weight'].iloc[i-1]) * 
            gp.quicksum(x[(i, j, k)] for j in range(1, len(location_codes)) if (i, j, k) in x)
            for i in range(1, len(location_codes)) # Skipping index 0 as it's depot
        ) <= truck_max_weight * y[k],
        f"Demand_{k}"
    )
# Each vehicle should leave the depot at least once
for k in truck_capacities:
    model.addConstr(
        gp.quicksum(x[(0, j, k)] for j in range(1, len(location_codes)) if (0, j, k) in x ) == 1,
        f"Leave_Depot_{k}"
    )

# Each vehicle should arrive at the depot at least once
for k in truck_capacities:
    model.addConstr(
        gp.quicksum(x[(i, 0, k)] for i in range(1, len(location_codes)) if (i, 0, k) in x) == 1,
        f"Return_Depot_{k}"
    )

# Ensure each customer is visited exactly once (excluding depot)
for i in range(1, len(location_codes)):  # Exclude depot index 0
    model.addConstr(
        gp.quicksum(
            x[(i, j, k)] for j in range(1, len(location_codes))  # Exclude depot index 0
            for k in truck_capacities if (i, j, k) in x
        ) == y[k],
        f"Visit_Customer_{i}"
    )

# # Time window constraints 
# for i in range(len(location_codes)):
#     for k in truck_capacities:
#         start_window = locations_df.loc[str(loc_df.iloc[i]), 'start_minutes'].item()
#         end_window = locations_df.loc[str(loc_df.iloc[i]), 'end_minutes'].item()
#         model.addConstr(t[(i, k)] >= start_window, f"Start_Window_{i}_{k}")
#         model.addConstr(t[(i, k)] <= end_window, f"End_Window_{i}_{k}")

big_M = 1e5  # A large number to effectively deactivate constraints for unused vehicles

# Time window constraints
for i in range(len(location_codes)):
    for k in truck_capacities:
        start_window = locations_df.loc[str(loc_df.iloc[i]), 'start_minutes'].item()
        end_window = locations_df.loc[str(loc_df.iloc[i]), 'end_minutes'].item()
        model.addConstr(t[(i, k)] >= start_window - big_M * (1 - y[k]), f"Start_Window_{i}_{k}")
        model.addConstr(t[(i, k)] <= end_window + big_M * (1 - y[k]), f"End_Window_{i}_{k}")


service_time_customer = 20  # minutes at customer locations
service_time_depot = 60  # minutes at the depot/warehouse

# for k in truck_capacities:
#     for i in range(len(location_codes)):
#         for j in range(1,len(location_codes)):
#             if i != j and (i, j, k) in x:
                
#                 travel_time = travel_matrix.get((loc_df.iloc[i], loc_df.iloc[j]), {}).get('travel_time_in_min', 0)
#                 # travel_time = travel_matrix.get((i, j), {}).get('travel_time_in_min', 0)
                
#                 # Check if i is the depot/warehouse (assuming the depot is labeled with a specific code, e.g., 'depot')
#                 if i == 0:
#                     service_time = service_time_depot
#                 else:
#                     service_time = service_time_customer
                
#                 # Constraint for time at each destination including travel and service times
#                 model.addConstr(
#                     t[(j, k)] >= t[(i, k)] + service_time + travel_time - 1e5 * (1 - x[(i, j, k)]),
#                     f"Service_Time_{i}_{j}_{k}"
#                 )

# Service time constraints
for k in truck_capacities:
    for i in range(len(location_codes)):
        for j in range(len(location_codes)):
            if i != j and (i, j, k) in x:
                travel_time = travel_matrix.get((loc_df.iloc[i], loc_df.iloc[j]), {}).get('travel_time_in_min', 0)
                if i == 0:
                    service_time = service_time_depot
                else:
                    service_time = service_time_customer
                
                model.addConstr(
                    t[(j, k)] >= t[(i, k)] + service_time + travel_time - big_M * (1 - x[(i, j, k)]),
                    f"Service_Time_{i}_{j}_{k}"
                )



# Linking constraint
for k in truck_capacities:
    for i in range(len(location_codes)):
        for j in range(len(location_codes)):
            if (i, j, k) in x:
                model.addConstr(y[k] >= x[(i, j, k)], f"Linking_{i}_{j}_{k}")

# Optimize the model
model.update()
model.optimize()
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible.")
    model.computeIIS()
    model.write("infeasible_mode1.ilp")
else:
    print("Optimal solution found.")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-05
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 4600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 25796 rows, 12863 columns and 111270 nonzeros
Model fingerprint: 0x46cc2b39
Variable types: 494 continuous, 12369 integer (12369 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [6e+03, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 13339 rows and 19 columns
Presolve time: 0.21s
Presolved: 12457 rows, 12844 columns, 72200 nonzeros
Variable types: 494 continuous, 12350 integer (12350 binary)
Found heuristic solution: objective 1.620267e+07

Root relaxation: objective 4.425716e+06, 134 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current No

In [24]:
for i in range(len(location_codes)):
    for j in range(len(location_codes)):
        for k in truck_capacities:
            if (i, j, k) in x and x[(i, j, k)].X > 0:
                print(f"x_{i}_{j}_{k} = {x[(i, j, k)].X}")



x_0_1_T3_2 = 1.0
x_0_1_T3_3 = 1.0
x_0_1_T3_4 = 1.0
x_0_1_T3_5 = 1.0
x_0_1_T7_1 = 1.0
x_0_1_T7_2 = 1.0
x_0_1_T10_3 = 1.0
x_0_1_T10_5 = 1.0
x_0_1_T10_6 = 1.0
x_0_1_T40_3 = 1.0
x_0_8_T10_1 = 1.0
x_0_9_T40_1 = 1.0
x_0_12_T10_7 = 1.0
x_0_15_T3_1 = 1.0
x_0_18_T3_6 = 1.0
x_0_18_T10_4 = 1.0
x_0_20_T40_2 = 1.0
x_0_24_T10_2 = 1.0
x_0_24_T40_4 = 1.0
x_1_0_T10_7 = 1.0
x_1_7_T40_4 = 1.0
x_2_4_T40_4 = 1.0
x_3_6_T40_4 = 1.0
x_4_2_T40_1 = 1.0
x_5_19_T40_3 = 1.0
x_6_3_T40_1 = 1.0
x_7_8_T40_1 = 1.0
x_8_1_T40_1 = 1.0
x_9_0_T10_6 = 1.0
x_9_24_T40_4 = 1.0
x_10_20_T40_2 = 1.0
x_11_0_T10_3 = 1.0
x_11_17_T40_4 = 1.0
x_12_21_T7_1 = 1.0
x_13_14_T3_1 = 1.0
x_14_16_T40_1 = 1.0
x_15_0_T40_1 = 1.0
x_15_0_T40_3 = 1.0
x_15_0_T40_4 = 1.0
x_15_23_T10_3 = 1.0
x_16_13_T40_1 = 1.0
x_17_0_T3_4 = 1.0
x_17_0_T7_2 = 1.0
x_17_0_T10_1 = 1.0
x_17_0_T10_2 = 1.0
x_17_0_T40_2 = 1.0
x_17_11_T40_1 = 1.0
x_18_0_T3_1 = 1.0
x_18_0_T3_2 = 1.0
x_18_0_T7_1 = 1.0
x_18_0_T10_5 = 1.0
x_18_9_T40_1 = 1.0
x_19_0_T10_4 = 1.0
x_19_5_T40_1 = 1.0
x_

In [25]:
# Define allowed vehicles
allowed_vehicles = {}
for i in range(len(location_codes)):
    for j in range(i + 1, len(location_codes)):
        trucks_i = locations_dff[str(loc_df.iloc[i])]['trucks_allowed']
        trucks_j = locations_dff[str(loc_df.iloc[j])]['trucks_allowed']
        common_truck_types = trucks_i.intersection(trucks_j)
        common_truck_ids = []
        for truck_type in common_truck_types:
            common_truck_ids.extend(type_to_ids.get(truck_type, []))
        allowed_vehicles[(str(loc_df.iloc[i]), str(loc_df.iloc[j]))] = common_truck_ids

In [26]:
# model2

In [29]:
# Create the model
model = gp.Model("CVRPTW2")

# Create decision variables
x = {}
t = {}
y = {}
for k in truck_capacities:
    y[k] = model.addVar(vtype=GRB.BINARY, name=f'y_{k}')
    for i in location_codes:
        for j in location_codes:
            if i != j: 
                x[(i, j, k)] = model.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}_{k}')
        if i in location_codes:
            t[(i, k)] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f't_{i}_{k}')

# Objective function: Minimize total distance and fixed costs
obj = gp.quicksum(
    travel_matrix.get((i, j), {}).get('travel_distance_in_km', 0) * x[(i, j, k)] * (20000 - int(truck_capacities[k]) / 1000)
    for k in truck_capacities
    for i in location_codes
    for j in location_codes if (i, j, k) in x and i != depot1 and j != depot1
) + gp.quicksum(
    int(truck_capacities[k]) * 2 * y[k]
    for k in truck_capacities
)
model.setObjective(obj, GRB.MINIMIZE)

# Flow balancing constraint
for i in location_codes:
    model.addConstr(
        gp.quicksum(x[(i, j, k)] for j in location_codes for k in truck_capacities if i != j and (i, j, k) in x) ==
        gp.quicksum(x[(j, i, k)] for j in location_codes for k in truck_capacities if i != j and (j, i, k) in x),
        f"Flow_Balancing_{i}"
    )

# Demand constraint
for k in truck_capacities:
    truck_max_weight = int(truck_capacities[k])
    model.addConstr(
        gp.quicksum(
            demands_dict.get(i, 0) * gp.quicksum(x[(i, j, k)] for j in order_destinations if (i, j, k) in x)
            for i in order_destinations
        ) <= truck_max_weight * y[k],
        f"Demand_{k}"
    )

# Ensure that each vehicle that leaves the depot returns to the depot
for k in truck_capacities:
    model.addConstr(
        gp.quicksum(x[(depot1, j, k)] for j in order_destinations if (depot1, j, k) in x) ==
        gp.quicksum(x[(j, depot1, k)] for j in order_destinations if (j, depot1, k) in x),
        f"Return_To_Depot_{k}"
    )

# Each customer must be visited exactly once
for i in order_destinations:
    model.addConstr(
        gp.quicksum(
            x[(i, j, k)] for j in location_codes for k in truck_capacities if (i, j, k) in x
        ) == 1,
        f"Visit_Customer_{i}"
    )

# Time window constraints 
for i in order_destinations:
    for k in truck_capacities:
        start_window = locations_df.loc[locations_df.index == str(i), 'start_minutes'].values[0]
        end_window = locations_df.loc[locations_df.index == str(i), 'end_minutes'].values[0]
        model.addConstr(t[(i, k)] >= start_window, f"Start_Window_{i}_{k}")
        model.addConstr(t[(i, k)] <= end_window, f"End_Window_{i}_{k}")

# Service time and travel time constraints
service_time_customer = 20  # minutes
service_time_depot = 60  # minutes
for k in truck_capacities:
    for i in order_destinations:
        for j in location_codes:
            if i != j and (i, j, k) in x:
                travel_time = travel_matrix.get((i, j), {}).get('travel_time_in_min', 0)
                service_time = service_time_customer if i in order_destinations else service_time_depot
                model.addConstr(
                    t[(j, k)] >= t[(i, k)] + service_time + travel_time - 1e5 * (1 - x[(i, j, k)]),
                    f"Service_Time_{i}_{j}_{k}"
                )





# Linking constraint
for k in truck_capacities:
    for i in location_codes:
        for j in location_codes:
            if (i, j, k) in x:
                model.addConstr(y[k] >= x[(i, j, k)], f"Linking_{i}_{j}_{k}")

# Optimize the model
model.update()
model.optimize()
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible.")
    model.computeIIS()
    model.write("infeasible_mode1.ilp")
else:
    print("Optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 4600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 25036 rows, 12749 columns and 102524 nonzeros
Model fingerprint: 0x7dcd4930
Variable types: 494 continuous, 12255 integer (12255 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [6e+03, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 11122 rows and 5035 columns
Presolve time: 0.28s
Presolved: 13914 rows, 7714 columns, 55347 nonzeros
Variable types: 361 continuous, 7353 integer (7353 binary)
Found heuristic solution: objective 128400.00000
Found heuristic solution: objective 5600.0000000

Explored 0 nodes (0 simplex iterations) in 0.45 seconds (0.18 work units)
Thread count was 12 (of 12 available processors)

Solution count 2: 5600 128400 

Opti

In [30]:
# print(model.printAttr('X'))

In [31]:

for i in location_codes:
    for j in location_codes:
        for k in truck_capacities:
            if (i, j, k) in x and x[(i, j, k)].X > 0:
                print(f"x_{i}_{j}_{k} = {x[(i, j, k)].X}")



x_A123_12854121_T3_1 = 1.0
x_A123_12854171_T3_1 = 1.0
x_A123_12854197_T3_1 = 1.0
x_A123_12854137_T3_1 = 1.0
x_A123_12854133_T3_1 = 1.0
x_A123_12854159_T3_1 = 1.0
x_A123_12854127_T3_1 = 1.0
x_A123_12983715_T3_1 = 1.0
x_A123_11950159_T3_1 = 1.0
x_A123_12838671_T3_1 = 1.0
x_A123_11646201_T3_1 = 1.0
x_A123_10208730_T3_1 = 1.0
x_A123_10214844_T3_1 = 1.0
x_A123_10214844_T3_1 = 1.0
x_A123_10208746_T3_1 = 1.0
x_A123_12738762_T3_1 = 1.0
x_A123_10208117_T3_1 = 1.0
x_A123_12171444_T3_1 = 1.0
x_A123_13015040_T3_1 = 1.0
x_A123_13067657_T3_1 = 1.0
x_A123_10208730_T3_1 = 1.0
x_A123_10208120_T3_1 = 1.0
x_A123_10208746_T3_1 = 1.0
x_A123_13005196_T3_1 = 1.0
x_A123_12769191_T3_1 = 1.0
x_12854121_A123_T3_1 = 1.0
x_12854171_A123_T3_1 = 1.0
x_12854197_A123_T3_1 = 1.0
x_12854137_A123_T3_1 = 1.0
x_12854133_A123_T3_1 = 1.0
x_12854159_A123_T3_1 = 1.0
x_12854127_A123_T3_1 = 1.0
x_12983715_A123_T3_1 = 1.0
x_11950159_A123_T3_1 = 1.0
x_12838671_A123_T3_1 = 1.0
x_11646201_A123_T3_1 = 1.0
x_10208730_A123_T3_1 = 1.0
x